In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install faiss-cpu sentence-transformers pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [4]:
# Đọc dữ liệu
df = pd.read_excel("/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/data_mogi_cleaned.xlsx")

# Xem dữ liệu
df.head()


,Đường,Phường/Xã,Quận/Huyện,Thành phố/Tỉnh,Giá,Diện tích(m2),Phòng ngủ,Phòng tắm,Pháp lý,Loại bđs,Giá/m^2
0,tên lửa,phường bình trị đông b,quận bình tân,tphcm,9700000000,221,3,2,sổ hồng,khác,4.389140e+07
1,nguyễn tuyển,phường bình trưng tây,quận 2 tp thủ đức,tphcm,22000000000,177,7,4,sổ hồng,mặt tiền,1.242938e+08
2,nguyễn duy trinh,phường bình trưng đông,quận 2 tp thủ đức,tphcm,14000000000,328,5,3,sổ hồng,biệt thự,4.268293e+07
3,trịnh như khuê,xã bình chánh,huyện bình chánh,tphcm,790000000,139,3,2,sổ hồng,nhà,5.683453e+06
4,huỳnh văn trí,xã bình chánh,huyện bình chánh,tphcm,720000000,167,3,2,sổ hồng,nhà,4.311377e+06


In [5]:
# Chia thành 5 phần
num_parts = 5
chunk_size = len(df) // num_parts

for i in range(num_parts):
    start = i * chunk_size
    end = None if i == num_parts - 1 else (i + 1) * chunk_size
    df_chunk = df.iloc[start:end]
    df_chunk.to_csv(f'/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_{i+1}.csv', index=False)
    print(f"Đã lưu: real_estate_part_{i+1}.csv")

Đã lưu: real_estate_part_1.csv
Đã lưu: real_estate_part_2.csv
Đã lưu: real_estate_part_3.csv
Đã lưu: real_estate_part_4.csv
Đã lưu: real_estate_part_5.csv


In [7]:
def format_row(row):
    return (
        f"Nhà ở {row['Đường']}, {row['Phường/Xã']}, {row['Quận/Huyện']}, {row['Thành phố/Tỉnh']}, "
        f"diện tích {row['Diện tích(m2)']}m², "
        f"{row['Phòng ngủ']} phòng ngủ, {row['Phòng tắm']} phòng tắm, "
        f"giá {row['Giá']} VND, "
        f"loại: {row['Loại bđs']}, pháp lý: {row['Pháp lý']}."
    )


Chia nhỏ file và chạy encoder cho từng file

In [8]:
model = SentenceTransformer("BAAI/bge-base-en-v1.5")
df = pd.read_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_1.csv')
df['text'] = df.apply(format_row, axis=1)
embeddings = model.encode(df['text'].tolist(), batch_size=64, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

faiss.write_index(index, '/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_1.faiss')
df.to_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_1_text.csv', index=False)
print("Đã xử lý xong part 1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/863 [00:00<?, ?it/s]

Đã xử lý xong part 1


In [9]:
df = pd.read_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_2.csv')

df['text'] = df.apply(format_row, axis=1)
embeddings = model.encode(df['text'].tolist(), batch_size=64, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

faiss.write_index(index, '/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_2.faiss')
df.to_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_2_text.csv', index=False)
print("Đã xử lý xong part 2")

Batches:   0%|          | 0/863 [00:00<?, ?it/s]

Đã xử lý xong part 2


In [10]:
df = pd.read_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_3.csv')

df['text'] = df.apply(format_row, axis=1)
embeddings = model.encode(df['text'].tolist(), batch_size=64, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

faiss.write_index(index, '/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_3.faiss')
df.to_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_3_text.csv', index=False)
print("Đã xử lý xong part 3")

Batches:   0%|          | 0/863 [00:00<?, ?it/s]

Đã xử lý xong part 3


In [11]:
df = pd.read_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_4.csv')

df['text'] = df.apply(format_row, axis=1)
embeddings = model.encode(df['text'].tolist(), batch_size=64, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

faiss.write_index(index, '/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_4.faiss')
df.to_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_4_text.csv', index=False)
print("Đã xử lý xong part 4")

Batches:   0%|          | 0/863 [00:00<?, ?it/s]

Đã xử lý xong part 4


In [12]:
df = pd.read_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_5.csv')

df['text'] = df.apply(format_row, axis=1)
embeddings = model.encode(df['text'].tolist(), batch_size=64, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

faiss.write_index(index, '/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_5.faiss')
df.to_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_5_text.csv', index=False)
print("Đã xử lý xong part 5")

Batches:   0%|          | 0/863 [00:00<?, ?it/s]

Đã xử lý xong part 5


In [14]:
# Tạo index trống cùng dimension
dimension = 768
combined_index = faiss.IndexFlatL2(dimension)

# Gộp từng phần
for i in range(1, 6):
    path = f"/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_{i}.faiss"
    part_index = faiss.read_index(path)

    # Lấy số lượng vector trong index con
    n_vectors = part_index.ntotal
    print(f"Gộp phần {i}: {n_vectors} vectors")

    # Trích vector ra & add vào index tổng
    xb = part_index.reconstruct_n(0, n_vectors)
    combined_index.add(xb)

# Lưu index gộp
faiss.write_index(combined_index, "/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_combined_index.faiss")
print("Đã gộp và lưu real_estate_combined_index.faiss thành công.")

Gộp phần 1: 55173 vectors
Gộp phần 2: 55173 vectors
Gộp phần 3: 55173 vectors
Gộp phần 4: 55173 vectors
Gộp phần 5: 55176 vectors
Đã gộp và lưu real_estate_combined_index.faiss thành công.


In [19]:
# Gộp tất cả các file part 1 → 5
combined_df = pd.read_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_1_text.csv')
for i in range(2, 6):
    path = f'/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_part_{i}_text.csv'
    part_df = pd.read_csv(path)
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)

# Lưu file gộp
combined_df.to_csv('/content/drive/MyDrive/trí tuệ nhân tạo /đồ án cơ sở /data/real_estate_combined_text.csv', index=False)
print("Đã gộp và lưu real_estate_combined_text.csv thành công.")

Đã gộp và lưu real_estate_combined_text.csv thành công.
